## Tesla and Waymo Data Presentation

In [1]:
import pandas as pd
import requests

### a. Download Tesla & Waymo Accident Datasets

In [2]:
def download_tesla_data():
    tesla_url = 'https://docs.google.com/spreadsheets/d/1ESnyJ4b7m96OCjs3GSQ6EGF7YOMuv0XV-ROXTYIazTs/export?format=csv'
    
    try:
        response = requests.get(tesla_url, timeout=10)
        response.raise_for_status()
        
        from io import StringIO
        tesla_df = pd.read_csv(StringIO(response.text), header=1)
        tesla_df.columns = tesla_df.columns.str.strip()
        return tesla_df
        
    except Exception as e:
        print(f"Tesla data download failed: {e}")
        raise SystemExit("Tesla dataset download failed")

def download_waymo_data():
    waymo_url = 'https://storage.googleapis.com/waymo-uploads/files/documents/safety/safety-impact-data/CSV2%20-%20Crashes%20with%20SGO%20ID%20and%20Group%20Membership%20202009-202503-2022benchmark.csv'
    
    try:        
        response = requests.get(waymo_url, timeout=10)
        response.raise_for_status()
        
        from io import StringIO
        waymo_df = pd.read_csv(StringIO(response.text))
        waymo_df.columns = waymo_df.columns.str.strip()
        return waymo_df
        
    except Exception as e:    
        print(f"Waymo data download failed: {e}")
        raise SystemExit("Waymo dataset download failed")

# Download data
tesla_df = download_tesla_data()
print(f"Tesla data: {len(tesla_df)} records")

waymo_df = download_waymo_data()
print(f"Waymo data: {len(waymo_df)} records")

Tesla data: 594 records
Waymo data: 723 records


### b. Data Preprocessing (standardization only)

In [3]:
def standardize_datasets():
    combined_data = []
    
    # Tesla data processing
    if 'tesla_df' in globals() and len(tesla_df) > 0:
        tesla_clean = pd.DataFrame()
        tesla_clean['case_id'] = tesla_df['Case #'].astype(str)
        tesla_clean['year'] = pd.to_numeric(tesla_df['Year'], errors='coerce').fillna(0).astype(int)
        tesla_clean['state'] = tesla_df['State'].fillna('Unknown')
        tesla_clean['deaths'] = pd.to_numeric(tesla_df['Deaths'], errors='coerce').fillna(0).astype(int)
        
        # Calculate total injuries
        injury_cols = ['Tesla driver', 'Tesla occupant', 'Other vehicle(s)', 'Cyclists/ Peds']
        total_injuries = 0
        for col in injury_cols:
            if col in tesla_df.columns:
                total_injuries += pd.to_numeric(tesla_df[col], errors='coerce').fillna(0)
        tesla_clean['injuries'] = total_injuries.astype(int)
        
        tesla_clean['autopilot'] = tesla_df['Autopilot claimed'].fillna(False)
        tesla_clean['source'] = 'Tesla'
        
        combined_data.append(tesla_clean)
        print(f"Tesla: {len(tesla_clean)} records")
    
    # Waymo data processing  
    if 'waymo_df' in globals() and len(waymo_df) > 0:
        waymo_clean = pd.DataFrame()
        waymo_clean['case_id'] = waymo_df['SGO Report ID'].astype(str)
        
        year_month = waymo_df['Year Month'].astype(str)
        waymo_clean['year'] = year_month.str[:4].astype(int)
        waymo_clean['state'] = waymo_df['Location'].str.split(',').str[-1].str.strip().fillna('Unknown')
        waymo_clean['deaths'] = 0  
        
        injuries = 0
        if 'Is Any-Injury-Reported' in waymo_df.columns:
            injuries += waymo_df['Is Any-Injury-Reported'].fillna(False).astype(int)
        waymo_clean['injuries'] = injuries
        
        waymo_clean['autopilot'] = True  
        waymo_clean['source'] = 'Waymo'
        
        combined_data.append(waymo_clean)
        print(f"Waymo: {len(waymo_clean)} records")
    
    # Combine datasets
    if combined_data:
        final_df = pd.concat(combined_data, ignore_index=True)
        
        final_df['year'] = final_df['year'].clip(2010, 2025)
        final_df['deaths'] = final_df['deaths'].clip(0, 20)
        final_df['injuries'] = final_df['injuries'].clip(0, 50)
        
        print(f"Combined dataset: {len(final_df)} records")
        return final_df
    else:
        print("No data to combine")
        return pd.DataFrame()

# Run standardization
print("Starting data standardization...")
standardized_data = standardize_datasets()

if len(standardized_data) > 0:
    print("Data standardization complete")
    globals()['clean_df'] = standardized_data
else:
    print("Failed to standardize data")

Starting data standardization...
Tesla: 594 records
Waymo: 723 records
Combined dataset: 1317 records
Data standardization complete


In [4]:
# EDA (Exploratory Data Analysis)

if 'standardized_data' not in globals() or len(standardized_data) == 0:
    print("No data available. Run preprocessing first.")
else:
    print(f"Dataset shape: {standardized_data.shape}")

def show_basic_info():
    print(f"Total records: {len(standardized_data)}")
    
    source_counts = standardized_data['source'].value_counts()
    print(f"Data sources:")
    for source, count in source_counts.items():
        percentage = count / len(standardized_data) * 100
        print(f"  {source}: {count} records ({percentage:.1f}%)")
    
    missing_counts = standardized_data.isnull().sum()
    has_missing = False
    for col, missing in missing_counts.items():
        if missing > 0:
            print(f"  {col}: {missing} missing")
            has_missing = True
    if not has_missing:
        print("  No missing values")

def analyze_temporal_patterns():
    year_counts = standardized_data['year'].value_counts().sort_index()
    print(f"Time range: {year_counts.index.min()} to {year_counts.index.max()}")
    
    print(f"Records by year:")
    for year, count in year_counts.items():
        print(f"  {year}: {count}")
    
    if 'autopilot' in standardized_data.columns:
        print(f"Autopilot usage by year:")
        
        df_clean = standardized_data.copy()
        df_clean['autopilot_bool'] = df_clean['autopilot'].map(lambda x: True if x in [True, 'True', 1, '1'] else False)
        
        autopilot_yearly = df_clean.groupby('year').agg({
            'case_id': 'count',
            'autopilot_bool': 'sum'
        })
        autopilot_yearly['autopilot_rate'] = (autopilot_yearly['autopilot_bool'] / autopilot_yearly['case_id']) * 100
        
        for year, row in autopilot_yearly.iterrows():
            print(f"  {year}: {int(row['autopilot_bool'])}/{row['case_id']} cases ({row['autopilot_rate']:.1f}%)")

def analyze_safety_outcomes():
    total_deaths = standardized_data['deaths'].sum()
    total_injuries = standardized_data['injuries'].sum()
    
    print(f"Total deaths: {total_deaths}")
    print(f"Total injuries: {total_injuries}")
    print(f"Average deaths per case: {standardized_data['deaths'].mean():.2f}")
    print(f"Average injuries per case: {standardized_data['injuries'].mean():.2f}")
    
    print(f"By data source:")
    for source in standardized_data['source'].unique():
        source_data = standardized_data[standardized_data['source'] == source]
        
        deaths_total = source_data['deaths'].sum()
        injuries_total = source_data['injuries'].sum()
        record_count = len(source_data)
        
        print(f"  {source}: {record_count} records, {deaths_total} deaths, {injuries_total} injuries")

def create_visualizations():
    try:
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        fig.suptitle('Autonomous Vehicle Safety Data Analysis', fontsize=16)
        
        # Records by year
        year_counts = standardized_data['year'].value_counts().sort_index()
        axes[0,0].bar(year_counts.index, year_counts.values, alpha=0.7, color='steelblue')
        axes[0,0].set_title('Records by Year')
        axes[0,0].set_xlabel('Year')
        axes[0,0].set_ylabel('Number of Records')
        axes[0,0].grid(True, alpha=0.3)
        
        # Data source distribution
        source_counts = standardized_data['source'].value_counts()
        axes[0,1].pie(source_counts.values, labels=source_counts.index, autopct='%1.1f%%', startangle=90)
        axes[0,1].set_title('Data Source Distribution')
        
        # Deaths vs injuries
        axes[1,0].scatter(standardized_data['deaths'], standardized_data['injuries'], 
                         alpha=0.6, color='coral')
        axes[1,0].set_xlabel('Deaths per Case')
        axes[1,0].set_ylabel('Injuries per Case')
        axes[1,0].set_title('Deaths vs Injuries Distribution')
        axes[1,0].grid(True, alpha=0.3)
        
        # Autopilot usage over time
        if 'autopilot' in standardized_data.columns:
            df_clean = standardized_data.copy()
            df_clean['autopilot_bool'] = df_clean['autopilot'].map(lambda x: True if x in [True, 'True', 1, '1'] else False)
            autopilot_by_year = df_clean.groupby('year')['autopilot_bool'].mean() * 100
            axes[1,1].plot(autopilot_by_year.index, autopilot_by_year.values, 
                          marker='o', linewidth=2, color='green')
            axes[1,1].set_xlabel('Year')
            axes[1,1].set_ylabel('Autopilot Usage (%)')
            axes[1,1].set_title('Autopilot Usage Trend')
            axes[1,1].grid(True, alpha=0.3)
        else:
            axes[1,1].text(0.5, 0.5, 'Autopilot data\nnot available', 
                          ha='center', va='center', transform=axes[1,1].transAxes, fontsize=12)
            axes[1,1].set_title('Autopilot Usage Trend')
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Visualization error: {e}")

# Run EDA
if 'standardized_data' in globals() and len(standardized_data) > 0:
    show_basic_info()
    analyze_temporal_patterns()
    analyze_safety_outcomes()
    create_visualizations()
else:
    print("No data available for EDA")

Dataset shape: (1317, 7)
Total records: 1317
Data sources:
  Waymo: 723 records (54.9%)
  Tesla: 594 records (45.1%)
  No missing values
Time range: 2010 to 2025
Records by year:
  2010: 14
  2013: 2
  2014: 4
  2015: 5
  2016: 15
  2017: 11
  2018: 18
  2019: 47
  2020: 40
  2021: 63
  2022: 100
  2023: 193
  2024: 540
  2025: 265
Autopilot usage by year:
  2010: 0/14.0 cases (0.0%)
  2013: 0/2.0 cases (0.0%)
  2014: 0/4.0 cases (0.0%)
  2015: 0/5.0 cases (0.0%)
  2016: 0/15.0 cases (0.0%)
  2017: 0/11.0 cases (0.0%)
  2018: 0/18.0 cases (0.0%)
  2019: 0/47.0 cases (0.0%)
  2020: 1/40.0 cases (2.5%)
  2021: 4/63.0 cases (6.3%)
  2022: 4/100.0 cases (4.0%)
  2023: 85/193.0 cases (44.0%)
  2024: 429/540.0 cases (79.4%)
  2025: 200/265.0 cases (75.5%)
Total deaths: 857
Total injuries: 1102
Average deaths per case: 0.65
Average injuries per case: 0.84
By data source:
  Tesla: 594 records, 857 deaths, 1038 injuries
  Waymo: 723 records, 0 deaths, 64 injuries
Visualization error: name 'plt'

### c. EDA (Exploratory Data Analysis)